# Portfolio Optimization
## Required libraries

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import minimize

## Tickers and Time Range

In [3]:
tickers = ['SPY','BND','GLD','QQQ','VTI']
end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)

## Download Adjusted Close Price

In [4]:
adj_close_df = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start = start_date,end = end_date, auto_adjust=True)
    adj_close_df[ticker] = data['Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Calculate Lognormal Returns

In [5]:
log_returns = np.log(adj_close_df)-np.log(adj_close_df.shift(1))
log_returns = log_returns.dropna()
print(log_returns)

                 SPY       BND       GLD       QQQ       VTI
Date                                                        
2020-01-22  0.000121  0.000828  0.000341  0.002639  0.000594
2020-01-23  0.001146  0.001771  0.002246  0.003211  0.001127
2020-01-24 -0.008933  0.001532  0.005829 -0.008451 -0.009530
2020-01-27 -0.016159  0.003292  0.006802 -0.020872 -0.015136
2020-01-28  0.010425 -0.001762 -0.008967  0.015243  0.009614
...              ...       ...       ...       ...       ...
2025-01-10 -0.015385 -0.005180  0.009513 -0.015805 -0.015130
2025-01-13  0.001549 -0.000983 -0.010001 -0.003219  0.001635
2025-01-14  0.001375  0.000702  0.005236 -0.000950  0.002256
2025-01-15  0.018027  0.007972  0.007461  0.022746  0.018175
2025-01-16 -0.001925  0.001948  0.006887 -0.007031 -0.000749

[1255 rows x 5 columns]


## Covariance Matrix

In [6]:
cov_matrix = log_returns.cov()*252
print(cov_matrix)

          SPY       BND       GLD       QQQ       VTI
SPY  0.044645  0.003227  0.005175  0.050730  0.045754
BND  0.003227  0.004962  0.003801  0.003873  0.003429
GLD  0.005175  0.003801  0.024219  0.006808  0.005399
QQQ  0.050730  0.003873  0.006808  0.066512  0.051894
VTI  0.045754  0.003429  0.005399  0.051894  0.047275


# Portfolio Performance Metrics

In [8]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

def expected_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)

## Risk Free Rate

In [14]:
risk_free_rate = .02

## Function to minimize

In [15]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
  return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

## Constraints, Bounds and Initial Weights

In [16]:
constaints = {"type" : "eq", "fun" : lambda weights : np.sum(weights)-1}
bounds = [(0, 0.4) for _ in range(len(tickers))]
initial_weights = np.array([1/len(tickers)]*len(tickers))

## Optimization

In [17]:
optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method="SLSQP", constraints=constaints, bounds=bounds)

optimal_weights = optimized_results.x

## Results

In [22]:

print("Optimal Weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

Optimal Weights:
SPY: 0.2000
BND: 0.0000
GLD: 0.4000
QQQ: 0.4000
VTI: 0.0000
Expected Annual Return: 0.1386
Expected Volatility: 0.1656
Sharpe Ratio: 0.7161
